In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report

In [2]:
path = './Bmp/'

In [3]:
train_data = []
for i, folder in enumerate(os.listdir('./Bmp/')):
    for image_name in os.listdir(os.path.join(path , folder)):
        train_data.append([path + folder+ "/" +image_name, i])

In [4]:
train_data_df = pd.DataFrame(train_data, columns = ['Image ID', 'Label'])

In [5]:
X = []
y = []
dsize = (28, 28)
for i in range(train_data_df.shape[0]):
    img = cv2.imread(train_data_df['Image ID'][i],  0)
    img = cv2.resize(img, dsize)
    X.append(img)
    y.append(train_data_df["Label"][i])

In [6]:
y = to_categorical(y)

In [7]:
X_train,X_test,y_train,y_test = train_test_split(np.array(X),np.array(y),test_size=0.33,random_state=17)

In [8]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
input_img = Input(shape=(28, 28, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same', kernel_initializer='he_uniform')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
flat = Flatten()(x)
den = Dense(512, activation='relu')(flat)
out = Dense(36, activation='softmax')(den)

In [10]:
classification_model = Model(input_img, out)

In [11]:
classification_model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
classification_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               3211776   
_________________________________________________________________
dense_1 (Dense)              (None, 36)                18468 

In [12]:
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.33,random_state=42)

In [13]:
history_2 = classification_model.fit(X_train, y_train, epochs=20,validation_data=(X_val,y_val))

Train on 2600 samples, validate on 1282 samples
Epoch 1/20
2600/2600 [==============================] - 10s 4ms/sample - loss: 3.3569 - accuracy: 0.1485 - val_loss: 2.8257 - val_accuracy: 0.3081
Epoch 2/20
2600/2600 [==============================] - 2s 697us/sample - loss: 1.9822 - accuracy: 0.5235 - val_loss: 1.5961 - val_accuracy: 0.5827
Epoch 3/20
2600/2600 [==============================] - 2s 580us/sample - loss: 0.9712 - accuracy: 0.7531 - val_loss: 1.0944 - val_accuracy: 0.7090
Epoch 4/20
2600/2600 [==============================] - 2s 753us/sample - loss: 0.5712 - accuracy: 0.8527 - val_loss: 1.0357 - val_accuracy: 0.7215
Epoch 5/20
2600/2600 [==============================] - 1s 566us/sample - loss: 0.3844 - accuracy: 0.9031 - val_loss: 0.9053 - val_accuracy: 0.7621
Epoch 6/20
2600/2600 [==============================] - 1s 502us/sample - loss: 0.2498 - accuracy: 0.9350 - val_loss: 0.9443 - val_accuracy: 0.7644
Epoch 7/20
2600/2600 [==============================] - 3s 973us/

In [14]:
y_predicted = classification_model.predict(X_test)

In [15]:
predicted_classes = np.argmax(np.round(y_predicted),axis=1)

In [16]:
y_test =  np.argmax(np.round(y_test),axis=1)

In [17]:
print(classification_report(y_test,predicted_classes))

              precision    recall  f1-score   support

           0       0.10      0.44      0.17        32
           1       0.62      0.48      0.54        21
           2       0.84      0.84      0.84        25
           3       0.75      0.86      0.80        14
           4       0.75      0.33      0.46        18
           5       0.88      0.64      0.74        22
           6       0.50      0.62      0.56        16
           7       1.00      0.61      0.76        18
           8       1.00      0.18      0.31        11
           9       0.88      0.50      0.64        14
          10       0.95      0.91      0.93       200
          11       0.65      0.39      0.49        38
          12       0.82      0.89      0.85        62
          13       0.78      0.85      0.81        66
          14       0.90      0.87      0.88       161
          15       0.74      0.90      0.81        29
          16       0.84      0.69      0.76        55
          17       0.76    

In [18]:
classification_model.save('my_model.h5') 